#### 환경설정

In [3]:
!pip install --quiet langchain
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [4]:
# 사용자 환경 설정
# modulabs통해 발급받은 API-KEY 사용
# 추후 open-ai, gemini KEY 발급시에도 아래처럼 환경 설정 가능

# 01 Colab용 비밀 키 불러오기
from google.colab import userdata
import os
from openai import OpenAI  # openai-python SDK

# 02 OpenAI API 키 설정
api_key = userdata.get("modulabsKey")  # Colab 사용자 환경변수 탭에서 등록한 키
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

# 03 버전 확인 (선택)
import openai
import langchain
print("LangChain ver:", langchain.__version__)
print("OpenAI SDK ver:", openai.__version__)

LangChain ver: 0.3.26
OpenAI SDK ver: 1.86.0


#### 01 Character TextSplitter
- CharacterTextSplitter는 Document객체를 단순히 주어진 글자 수만큼 여러 개의 청크로 분할

In [5]:
# Langchain Text Splitter 모듈 다운로드
!pip install -qU langchain-text-splitters

In [7]:
!pip install --quiet pypdfium2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.1 MB/s eta 0:00:00


In [8]:
# PyPDFium2Loader로 PDF 문서 로드하기
# colab 환경이므로 mount 필요

from google.colab import drive
drive.mount('/content/drive')

from langchain_community.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

# CharacterTextSplitter 모듈 로드
from langchain_text_splitters import CharacterTextSplitter

# 구분자: 줄 바꿈, 청크 길이: 500, 청크 오버랩 : 100, length_function : 글자 수

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len
)

# 텍스트 분할
texts = text_splitter.split_documents(pages)
print(texts[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로
 CIS이슈리포트 2022-2호 | 1 |
<요 약>
▶ 혁신성장 정책금융기관*은 혁신성장산업 영위기업을 발굴·지원하기 위한 정책금융 가이드라인**에 따라 혁신
성장 기술분야에 대한 금융지원을 강화하고 있음
 * 산업은행, 기업은행, 수출입은행, 신용보증기금, 기술보증기금, 중소벤처기업진흥공단, 무역보험공사 등 
11개 기관
 ** 혁신성장 정책금융 지원 대상을 판단하는 기준으로, ‘9대 테마 – 46개 분야 – 296개 품목’으로 구성
￮ 정책금융기관의 혁신성장 정책금융 공급규모는 2017년 24.1조 원에서 2021년 85.4조 원으로 크게 증가
하여 국내 산업 구조의 미래 산업으로의 전환을 충실히 지원하고 있음
￮ 본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의
기술·시장 동향을 분석함' metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'title': '', 'author': 'kmd kdy', 'subject': '', 'keywords': '', 'moddate': '2022-07-29T09:03:16+09:00', 'source': '/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18, 'page': 0}


/usr/local/lib/python3.11/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [9]:
print(texts[1])

page_content='￮ 본 보고서는 ICT 산업의 정책금융 지원 트렌드를 파악하고, 혁신성장 정책금융이 집중되는 주요 품목의
기술·시장 동향을 분석함
▶ 혁신성장 ICT 산업은 정보통신(6개 분야, 47개 품목), 전기전자(5개 분야, 27개 품목), 센서측정(3개 분야, 19개 품목) 테마로 구성되며, 혁신성장 정책금융기관의 공급액 규모는 2021년 말 기준 16.9조 원으로 
2017년 이후 연평균 39.2% 지속 증가하고 있음
￮ ICT 산업의 공급액 규모 비중은 혁신성장 정책금융 총 공급 규모의 약 20% 수준임
 * (‘17)18.7% → (’18)20.7% → (’19)18.5% → (’20)20.3% → (’21)19.8%
혁신성장 정책금융 동향 :
ICT 산업을 중심으로
오동찬 선임조사역 (dcoh@kcredit.or.kr)' metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2022-07-29T09:03:16+09:00', 'title': '', 'author': 'kmd kdy', 'subject': '', 'keywords': '', 'moddate': '2022-07-29T09:03:16+09:00', 'source': '/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'total_pages': 18, 'page': 0}


In [10]:
from langchain_community.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

pages = loader.load()

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)

print([len(i.page_content) for i in texts ])

[784, 1372, 1156, 1195, 682, 655, 1550, 550, 1353, 1277, 1076, 531, 1245, 1094, 1293, 920, 956, 1034]


/usr/local/lib/python3.11/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


#### 02 RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 여러 개의 구분자를 재귀적으로 적용하여 텍스트 분할
- Chunkviz 웹 페이지 이용하면, CharacterTextSplitter와 RecursiveCharacterTextSplitter의 실제 수행 결과를 자세히 확인 가능

In [12]:
from langchain_community.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

pages = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n"," ",""],
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)

print([len(i.page_content) for i in texts ])

[449, 407, 485, 496, 477, 151, 466, 484, 334, 467, 454, 427, 492, 271, 473, 230, 485, 456, 455, 347, 481, 144, 496, 494, 488, 141, 479, 474, 456, 497, 485, 274, 478, 107, 482, 480, 410, 463, 484, 294, 490, 484, 453, 478, 498, 495, 485, 82, 493, 490, 228]


/usr/local/lib/python3.11/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


#### 03 SemanticChunker
- 앞서 소개한 두 개의(CharacterTextSplitter, RecursiveCharacterTextSplitter) Text Splitter는 사용자가 지정한 청크 길이 기반으로 문서 분할
- 다만 상기 두 방식은 기계적으로 문서 분할한다는 단점이 있음

In [13]:
!pip install --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 2.9 MB/s eta 0:00:00


In [16]:
!pip install --quiet pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 4.2 MB/s eta 0:00:00


In [19]:
!pip install --quiet langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.6 MB/s eta 0:00:00


In [20]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/BOK 이슈노트 제2022-38호 인공지능 언어모형을 이용한 인플레이션 어조지수 개발 및 시사점.pdf")
pages = loader.load()

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from google.colab import userdata

api_key = userdata.get("modulabsKey")
text_splitters = SemanticChunker(OpenAIEmbeddings(openai_api_key = api_key))

texts = text_splitter.split_documents(pages)
print("-"*100)
print("[첫 번째 청크]")
print(texts[0].page_content)
print("-"*100)
print("[두 번째 청크]")

print(texts[1].page_content)

----------------------------------------------------------------------------------------------------
[첫 번째 청크]
1
인공지능 언어모형을 이용한 인플레이션 어조지수 개발 및 시사점
뉴스기사, 소셜미디어와 같은 텍스트 데이터는 크게 두 가지 경로를 
통해 경제 분석에 유용한 정보를 제공할 수 있다. 첫째, 지정학적 리스
크, 이상기후 등 이벤트는 수입물가지수와 같은 공식통계에 앞서 뉴스 
속보나 인터넷 커뮤니티에 먼저 반영되며 향후 인플레이션 수준에 대한 
정보를 제공한다. 둘째, 거시경제 여건, 기업의 가격설정 행태 등에 대
한 언론 보도는 기업의 인플레이션 기대를 변화시킴으로써 의사결정에 
영향을 미칠 수 있다. 그러나, 이러한 중요성에도 불구하고, 텍스트 데
이터는 계량화가 쉽지 않아 경제 분석에 활용하는 데 어려움이 크다는 
평가도 있다(로버트 쉴러, 2013년 노벨 경제학상 수상).
최근 정보통신 및 인공지능 기술과 함께 텍스트 분석기법이 빠르게 
발전하면서 경제 분석에 텍스트를 활용하려는 시도가 점차 늘고 있다. 
대표적으로 뉴스기사에 반영된 경제 불확실성을 측정하거나 중앙은행
----------------------------------------------------------------------------------------------------
[두 번째 청크]
발전하면서 경제 분석에 텍스트를 활용하려는 시도가 점차 늘고 있다. 
대표적으로 뉴스기사에 반영된 경제 불확실성을 측정하거나 중앙은행 
회의록에 반영된 통화정책 기조를 평가한 연구가 있으며, 각국 중앙은
행도 텍스트에 반영된 경제주체의 심리를 파악하여 경기상황을 판단하
거나 전망하기 위해 노력하고 있다. 이들 연구는 문서내 단어별 빈도를 
계산하는 간단한 방법부터 단어의 문맥적 의미를 파악하는 딥러닝 알
고리즘까지 다양한 분석기법을 활용하고 있다. 이 가운데, 최근 특히 
주목받는 분석기법으로 인공지

In [21]:
print([len(i.page_content) for i in texts])

[495, 491, 498, 497, 469, 138, 485, 479, 489, 453, 354, 495, 468, 497, 401, 365, 481, 482, 478, 439, 254, 498, 494, 478, 448, 390, 490, 478, 496, 472, 248, 486, 479, 498, 195, 486, 463, 493, 423, 234, 486, 479, 476, 225, 490, 478, 440, 486, 496, 324, 500, 496, 475, 464, 470, 495, 375, 495, 446, 498, 345, 457, 491, 436, 461, 436, 143, 444, 475, 437, 485, 487, 461]
